# VacationPy - Aaron Wollman
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
import sys
sys.path.append("..")
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = "../WeatherPy/output_data/cities.csv"
city_data = pd.read_csv(filepath)
city_data.head()

,City,country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Pitimbu,BR,-7.47,-34.81,75.20,83,49,10.29
1,Kenai,US,60.55,-151.26,55.99,87,90,3.36
2,Bowling Green,US,36.99,-86.44,84.20,66,90,8.05
3,New Norfolk,AU,-42.78,147.06,42.01,83,100,2.86
4,Kavaratti,IN,10.57,72.64,80.01,85,100,17.96


In [3]:
class City_Weather_Columns:
    city = "City"
    lat = "Lat"
    lng = "Lng"
    max_temp = "Max Temp"
    humidity = "Humidity"
    cloudiness = "Cloudiness"
    wind_speed = "Wind Speed"
    country = "country"

In [4]:
# Fix data types
numeric_columns = [
    City_Weather_Columns.lat,
    City_Weather_Columns.lng,
    City_Weather_Columns.max_temp,
    City_Weather_Columns.cloudiness,
    City_Weather_Columns.wind_speed
] 
for col in numeric_columns:
    city_data[col] = city_data[col].astype("float")

city_data.head()

,City,country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Pitimbu,BR,-7.47,-34.81,75.20,83,49.0,10.29
1,Kenai,US,60.55,-151.26,55.99,87,90.0,3.36
2,Bowling Green,US,36.99,-86.44,84.20,66,90.0,8.05
3,New Norfolk,AU,-42.78,147.06,42.01,83,100.0,2.86
4,Kavaratti,IN,10.57,72.64,80.01,85,100.0,17.96


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure GMaps
gmaps.configure(api_key=g_key)

In [6]:
# Get Locations
location_cols = [
    City_Weather_Columns.lat,
    City_Weather_Columns.lng
]
locations = city_data[location_cols]
locations.head()

,Lat,Lng
0,-7.47,-34.81
1,60.55,-151.26
2,36.99,-86.44
3,-42.78,147.06
4,10.57,72.64


In [7]:
# Get Humidity
humidity = city_data[City_Weather_Columns.humidity]
humidity.head()

0    83
1    87
2    66
3    83
4    85
Name: Humidity, dtype: int64

In [8]:
# Display Heatmap
def get_heat_layer(locations, humidity):
    return gmaps.heatmap_layer(locations, weights = humidity, \
                                dissipating=False, max_intensity = 100, \
                                point_radius = 3)

fig = gmaps.figure()

heat_layer = get_heat_layer(locations, humidity)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
nice_weather = city_data.loc[city_data[City_Weather_Columns.max_temp] < 80]
nice_weather = nice_weather[nice_weather[City_Weather_Columns.max_temp] > 70]
nice_weather = nice_weather[nice_weather[City_Weather_Columns.cloudiness] == 0]
nice_weather = nice_weather.drop(columns = City_Weather_Columns.cloudiness)
nice_weather = nice_weather.dropna()
nice_weather.head()

,City,country,Lat,Lng,Max Temp,Humidity,Wind Speed
23,Rosetta,EG,31.40,30.42,78.80,78,8.05
43,Huilong,CN,31.81,121.66,75.00,91,10.02
60,Jalu,LY,29.03,21.55,79.59,49,16.49
80,Birjand,IR,32.87,59.22,73.40,28,6.93
96,Ýpsonas,CY,34.69,32.96,78.80,88,12.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Retrieve Hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
def get_hotel_json(lat, lng):
    coordinates = f"{lat}, {lng}"
    search_type = "hotels"
    radius = 5000 # meters
    search_keyword = "hotel"
    
    params = {
        "location" : coordinates,
        "radius" : radius,
        "type" : search_type,
        "key" : g_key,
        "keyword" : search_keyword
    }
    
    response = requests.get(base_url, params = params)
    hotels_json =response.json()
    
    return hotels_json["results"][0]

In [11]:
class Hotel_Columns:
    name = "Hotel Name"
    city = "City"
    country = "Country"
    lat = "Lat"
    lng = "Lng"

In [12]:
hotel_infos = []
for index, row in nice_weather.iterrows():
    lat, lng = row[location_cols]
    hotel_city = row[City_Weather_Columns.city]
    hotel_country = row[City_Weather_Columns.country]
    
    try:
        hotel = get_hotel_json(lat, lng)
        hotel_name = hotel["name"]
        hotel_info = {
            Hotel_Columns.name : hotel_name,
            Hotel_Columns.city : hotel_city,
            Hotel_Columns.country : hotel_country,
            Hotel_Columns.lat : lat,
            Hotel_Columns.lng : lng
        }
        hotel_infos.append(hotel_info)
        print(f"Found {hotel_name} in {hotel_city}, {hotel_country}.")
    except:
        print(f"Could not find hotel info for {hotel_city}, {hotel_country}.")

Found Mecca Hotel in Rosetta, EG.
Found Hilton Garden Inn Qidong in Huilong, CN.
Found Jalu Hotel in Jalu, LY.
Found Sepehr Hotel in Birjand, IR.
Found Villa Mosaica in Ýpsonas, CY.
Found Orlanova Hotel in Arraial do Cabo, BR.
Could not find hotel info for Sidi Ali, DZ.
Found Hotel Bagoeira in Barcelos, PT.
Could not find hotel info for Konya Province, TR.
Found ديلانا فرع 2 in Darnah, LY.
Found Six Senses Douro Valley in Seara, PT.
Could not find hotel info for Jadu, LY.
Found InterContinental Bora Bora Resort Thalasso Spa in Faanui, PF.
Found OYO Pousada Mediterrâneo in São João da Barra, BR.
Found Maitai Lapita Village Huahine in Fare, PF.
Found Sofitel Algiers Hamma Garden in Algiers, DZ.
Found 7 Days Inn in Kumul, CN.
Found Kokopo Beach Bungalow Resort in Kokopo, PG.
Found Hotel Jardines Baja in San Quintín, MX.
Found Palace of Zinos in Ponta do Sol, PT.
Found Hotel DiliMah Premium Luxury in Samarkand, UZ.
Found Graciosa Resort - Biosphere Island Hotel in Lagoa, PT.
Found Kokoda T

In [13]:
hotel_df = pd.DataFrame(hotel_infos)
hotel_df.head()

,Hotel Name,City,Country,Lat,Lng
0,Mecca Hotel,Rosetta,EG,31.40,30.42
1,Hilton Garden Inn Qidong,Huilong,CN,31.81,121.66
2,Jalu Hotel,Jalu,LY,29.03,21.55
3,Sepehr Hotel,Birjand,IR,32.87,59.22
4,Villa Mosaica,Ýpsonas,CY,34.69,32.96


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))